In [4]:
import pandas as pd
import gensim.models.keyedvectors as word2vec
from numpy.linalg import norm
import numpy as np
from scipy.spatial.distance import cosine
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

In [22]:
#df = pd.read_csv("./word2vec.csv",header=0, keep_default_na=False)
#df = pd.read_csv("./word2vec_debiased.csv",header=0, keep_default_na=False)
path = "../data/word_embeddings/"
#model =  word2vec.KeyedVectors.load_word2vec_format(path+'word2vec_50k.bin', binary=True)
model =  word2vec.KeyedVectors.load_word2vec_format(path+'glove_50k.bin', binary=True)

C:\Users\Bhavya\Anaconda3\envs\semantic\lib\site-packages\smart_open\smart_open_lib.py:251: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  warnings.warn(


In [23]:
df = pd.DataFrame({"word":list(model.vocab.keys())})
df.head()

,word
0,the
1,of
2,to
3,and
4,in


In [24]:
# calculate bias direction when we have group of words not pairs
def groupBiasDirection(gp1, gp2):
    #print(gp1,gp2)
    dim = len(model["he"])
    g1,g2 = np.zeros((dim,), dtype=float), np.zeros((dim,), dtype=float)
    for p in gp1:
        p = p.strip()
        if p not in model:
            continue
        p_vec = model[p]/norm(model[p])
        g1 = np.add(g1,p_vec)

    for q in gp2:
        q = q.strip()
        if q not in model:
            continue
        q_vec = model[q]/norm(model[q])
        g2 = np.add(g2,q_vec) 

    g1, g2 = g1/norm(g1), g2/norm(g2)
    return (g1,g2)

In [25]:
gender_bias = ["man,boy,he,father,son,guy,male,his,himself,john".split(","),
               "woman,girl,she,mother,daughter,gal,female,her,herself,mary".split(",")]
#eco_bias = [("rich","wealthy"),("poor","impoverished")]
race_bias = ["aisha,keisha,tamika,lakisha,tanisha,latoya,kenya,latonya,ebony,rasheed,tremayne,kareem,darnell,tyrone,hakim,jamal,leroy,jermaine".split(","),
             "emily,anne,jill,allison,laurie,sarah,meredith,carrie,kristen,todd,neil,geoffrey,brett,brendan,greg,matthew,jay,brad".split(",")]
religion_bias = ["baptism, messiah, catholicism, resurrection, christianity, salvation, protestant, gospel, trinity, jesus, christ, christian, cross, catholic, church".split(","),
                "allah, ramadan, turban, emir, salaam, sunni, koran, imam, sultan, prophet, veil, ayatollah, shiite, mosque, islam, sheik, muslim, muhammad".split(",")]
sentiment_bias = ["caress, freedom, health, love, peace, cheer, friend, heaven, loyal, pleasure, diamond, gentle, honest, lucky, rainbow, diploma, gift, honor, miracle, sunrise, family, happy, laughter, paradise, vacation".split(","),
                 "abuse, crash, filth, murder, sickness, accident, death, grief, poison, stink, assault, disaster, hatred, pollute, tragedy, divorce, jail, poverty, ugly, cancer, kill, rotten, vomit, agony, prison".split(",")]
age_bias = ["tiffany,michelle,cindy,kristy,brad,eric,joey,billy".split(","),
           "ethel,bernice,gertrude,agnes,cecil,wilbert,mortimer,edgar".split(",")]

eco_bias = ["rich,richer,richest,affluence,affluent,wealthy,costly,lavish,luxury,plush,expensive,invaluable".split(","),
           "poor,poorer,poorest,needy,impoverished,economical,inexpensive,cheap,bankrupt,worthless,basic,plain".split(",")]
bias_words = {"gender":gender_bias, "religion":religion_bias, "race":race_bias, "age":age_bias, "sentiment":sentiment_bias, "economic":eco_bias}

In [26]:
for w in age_bias[1]:
    if w not in model:
        print(w)

In [27]:
bias_words

{'gender': [['man',
   'boy',
   'he',
   'father',
   'son',
   'guy',
   'male',
   'his',
   'himself',
   'john'],
  ['woman',
   'girl',
   'she',
   'mother',
   'daughter',
   'gal',
   'female',
   'her',
   'herself',
   'mary']],
 'religion': [['baptism',
   ' messiah',
   ' catholicism',
   ' resurrection',
   ' christianity',
   ' salvation',
   ' protestant',
   ' gospel',
   ' trinity',
   ' jesus',
   ' christ',
   ' christian',
   ' cross',
   ' catholic',
   ' church'],
  ['allah',
   ' ramadan',
   ' turban',
   ' emir',
   ' salaam',
   ' sunni',
   ' koran',
   ' imam',
   ' sultan',
   ' prophet',
   ' veil',
   ' ayatollah',
   ' shiite',
   ' mosque',
   ' islam',
   ' sheik',
   ' muslim',
   ' muhammad']],
 'race': [['aisha',
   'keisha',
   'tamika',
   'lakisha',
   'tanisha',
   'latoya',
   'kenya',
   'latonya',
   'ebony',
   'rasheed',
   'tremayne',
   'kareem',
   'darnell',
   'tyrone',
   'hakim',
   'jamal',
   'leroy',
   'jermaine'],
  ['emily',
 

In [28]:
all_words = list(model.vocab.keys()) #[:50000]
#all_words = df["word"].tolist()
df = pd.DataFrame({"word":all_words})
for bias_type in bias_words:
    bias_w = bias_words[bias_type]
    df[bias_type] = None
    g1, g2 = groupBiasDirection(bias_w[0], bias_w[1])
    for index, row in df.iterrows():
        w = row["word"]
        # assuming group bias "Quantification algo"
        df.at[index, bias_type] = round(cosine(g1,model[w])-cosine(g2,model[w]),4)

In [29]:
gen_max, gen_min = df["gender"].max(), df["gender"].min()
sen_max, sen_min = df["sentiment"].max(), df["sentiment"].min()
race_max, race_min = df["race"].max(), df["race"].min()
relg_max, relg_min = df["religion"].max(), df["religion"].min()
age_max, age_min = df["age"].max(), df["age"].min()
eco_max, eco_min = df["economic"].max(), df["economic"].min()

print("Gender: ",gen_min,gen_max)
print("Sentiment: ",sen_min, sen_max)
print("Race: ",race_min, race_max)
print("Religion: ",relg_min, relg_max)
print("Age: ",age_min, age_max)
print("Economic: ",eco_min, eco_max)

Gender:  -0.3016 0.3069
Sentiment:  -0.4373 0.4408
Race:  -0.4841 0.5404
Religion:  -0.5434 0.5909
Age:  -0.4302 0.4968
Economic:  -0.4053 0.3214


In [16]:
# while calculating for negative values we want the range to be [-1, 0] instead of [0,1]
# so, we have used 'negative' parameter to flip the sign if negative values are fed 
def percentile_rank(values, col, negative=False):
    N = len(values)
    last_ind = -1
    for i,items in enumerate(values.iteritems()): 
        index, val = items[0], items[1]
        if last_ind!=-1 and val==df.at[last_ind, col]: 
            df.at[index, col] = df.at[last_ind, col] 
            #percentile.append(percentile[i-1])
        else:
            p = (N-i)/N
            #print(i,p)
            df.at[index, col] = p 
            #percentile.append(p)
        if negative:
            df.at[index, col] = df.at[index, col]*-1
        last_ind = index

for col in df.columns:
    if col=="word":
        continue
    values = df.loc[df[col]>0][col].sort_values(ascending=False, inplace=False)
    percentile_rank(values, col)
    
    values = df.loc[df[col]<0][col].sort_values(ascending=True, inplace=False)
    percentile_rank(values, col, negative=True)

In [30]:
# normalization of bias scores
for index, row in df.iterrows():
    if row["gender"]>0:
        df.at[index, "gender"] = row["gender"]/gen_max
    else:
        df.at[index, "gender"] = -1*row["gender"]/gen_min
        
    if row["race"]>0:
        df.at[index, "race"] = row["race"]/race_max
    else:
        df.at[index, "race"] = -1*row["race"]/race_min
    
    if row["sentiment"]>0:
        df.at[index, "sentiment"] = row["sentiment"]/sen_max
    else:
        df.at[index, "sentiment"] = -1*row["sentiment"]/sen_min
        
    if row["religion"]>0:
        df.at[index, "religion"] = row["religion"]/relg_max
    else:
        df.at[index, "religion"] = -1*row["religion"]/relg_min
    
    if row["age"]>0:
        df.at[index, "age"] = row["age"]/age_max
    else:
        df.at[index, "age"] = -1*row["age"]/age_min  
    
    if row["economic"]>0:
        df.at[index, "economic"] = row["economic"]/eco_max
    else:
        df.at[index, "economic"] = -1*row["economic"]/eco_min  

In [31]:
gen_max, gen_min = df["gender"].max(), df["gender"].min()
sen_max, sen_min = df["sentiment"].max(), df["sentiment"].min()
race_max, race_min = df["race"].max(), df["race"].min()
relg_max, relg_min = df["religion"].max(), df["religion"].min()
age_max, age_min = df["age"].max(), df["age"].min()
eco_max, eco_min = df["economic"].max(), df["economic"].min()

print("Gender: ",gen_min,gen_max)
print("Sentiment: ",sen_min, sen_max)
print("Race: ",race_min, race_max)
print("Religion: ",relg_min, relg_max)
print("Age: ",age_min, age_max)
print("Economic: ",eco_min, eco_max)

Gender:  -1.0 1.0
Sentiment:  -1.0 1.0
Race:  -1.0 1.0
Religion:  -1.0 1.0
Age:  -1.0 1.0
Economic:  -1.0 1.0


In [32]:
df.head()

,word,gender,religion,race,age,sentiment,economic
0,the,-0.306034,-0.169488,0.592524,-0.251278,-0.148182,0.186683
1,of,-0.349138,-0.215311,0.610844,-0.145049,-0.0523668,0.0973864
2,to,-0.28813,-0.142989,0.560511,-0.316597,-0.0841528,0.290292
3,and,-0.290451,-0.160655,0.626203,-0.387959,-0.0461925,0.133167
4,in,-0.367706,-0.158815,0.627683,-0.252441,0.0174682,0.0681394


In [33]:
df.describe()

,word,gender,religion,race,age,sentiment,economic
count,50024,50024.000000,50024.000000,50024.000000,50024.000000,50024.000000,50024.00000
unique,50024,3783.000000,5378.000000,6195.000000,4965.000000,5186.000000,3762.00000
top,autobot,-0.057692,0.054324,-0.194794,0.013688,-0.038418,-0.03923
freq,1,56.000000,39.000000,34.000000,36.000000,43.000000,47.00000


In [34]:
df.shape

(50024, 7)

In [35]:
# default option: min-max Normalization, _percentile option: percentile feature scaling

#df.to_csv("../data/word2vec_50k.csv", encoding='utf-8', index=False)
#df.to_csv("../data/word2vec_50k_percentile.csv", encoding='utf-8', index=False)
df.to_csv("../data/glove_50k.csv", encoding='utf-8', index=False)
#df.to_csv("../data/glove_50k_percentile.csv", encoding='utf-8', index=False)

## Miscellaneous

In [49]:
#v = [0.90, 0.87, 0.87, 0.76, 0.60, 0.32, 0.32, 0.32, 0.1, 0.05]
values = df.loc[df["gender"]>0]["gender"].sort_values(ascending=False, inplace=False)
values

8128      0.344
62       0.3336
42430    0.3316
57        0.312
16295    0.3069
          ...  
35457    0.0001
34752    0.0001
32733    0.0001
34231    0.0001
25589    0.0001
Name: gender, Length: 20221, dtype: object

In [39]:
percentile = []
N = len(values)
for i, val in enumerate(values):
    if val==values[i-1]:
        percentile.append(percentile[i-1])
        continue
    p = (N-i)/N*100
    print(i,p)
    percentile.append(p)

0 100.0
1 90.0
3 70.0
4 60.0
5 50.0
8 20.0
9 10.0


In [40]:
percentile

[100.0, 90.0, 90.0, 70.0, 60.0, 50.0, 50.0, 50.0, 20.0, 10.0]

In [47]:
df.loc[df["gender"]>0]["gender"].sort_values(ascending=False, inplace=False)

8128      0.344
62       0.3336
42430    0.3316
57        0.312
16295    0.3069
          ...  
35457    0.0001
34752    0.0001
32733    0.0001
34231    0.0001
25589    0.0001
Name: gender, Length: 20221, dtype: object

In [41]:
for col in df.columns:
    if col=="word":
        continue
    values = df.loc[df[col]>0][col].sort_values(ascending=False, inplace=False)

,word,gender,religion,race,age,sentiment
0,in,-0.0391,0.0022,0.0182,-0.0031,0.0433
1,for,-0.0491,-0.0273,-0.0058,-0.0341,-0.0399
2,that,-0.0621,-0.0591,0.0078,-0.0792,-0.0582
3,is,-0.0432,-0.0588,-0.0449,0.0088,-0.0266
4,on,-0.0459,-0.0029,0.0526,-0.0537,0.0032
...,...,...,...,...,...,...
50036,salaam,-0.005,0.2614,-0.1961,-0.1378,-0.1366
50037,sunni,-0.0334,0.2043,-0.1787,-0.1393,-0.0103
50038,koran,-0.0474,0.1698,-0.0362,-0.106,0.1036
50039,shiite,-0.0162,0.2177,-0.1113,-0.1006,0.1197


In [11]:
# normalization of bias scores
'''
for index, row in df.iterrows():
    if row["gender"]>0:
        df.at[index, "gender"] = row["gender"]/gen_max
    else:
        df.at[index, "gender"] = -1*row["gender"]/gen_min
        
    if row["race"]>0:
        df.at[index, "race"] = row["race"]/race_max
    else:
        df.at[index, "race"] = -1*row["race"]/race_min
    
    if row["sentiment"]>0:
        df.at[index, "sentiment"] = row["sentiment"]/sen_max
    else:
        df.at[index, "sentiment"] = -1*row["sentiment"]/sen_min
        
    if row["religion"]>0:
        df.at[index, "religion"] = row["religion"]/relg_max
    else:
        df.at[index, "religion"] = -1*row["religion"]/relg_min
    
    if row["age"]>0:
        df.at[index, "age"] = row["age"]/age_max
    else:
        df.at[index, "age"] = -1*row["age"]/age_min  
'''